In [1]:
import keras
import tensorflow as tf
import numpy
import pandas
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split

2022-01-15 19:57:30.790014: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-15 19:57:30.790029: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
dataset = pandas.read_csv("Dane1.csv", index_col=False)
dataset = dataset.iloc[:, 1:]
dataset["target"] = tf.keras.utils.to_categorical(dataset["target"])
train, test = train_test_split(dataset, test_size=0.2, shuffle = True, random_state=2137)
dataset.head()

,target,id,date,flag,username,tweet
0,1.0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1.0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,1.0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,1.0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,1.0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:

BUFFER_SIZE = 10000
BATCH_SIZE = 32

ds_train = tf.data.Dataset.from_tensor_slices((train["tweet"], train["target"]))
ds_train = ds_train.batch(32)
ds_train = ds_train.prefetch(32)
print(ds_train.element_spec)

ds_test = tf.data.Dataset.from_tensor_slices((test["tweet"], test["target"]))
ds_test = ds_test.batch(32)
ds_test = ds_test.prefetch(32)
print(ds_test.element_spec)

#ds_train = ds_train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
#ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)



(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))
(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))


2022-01-15 19:58:43.154512: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-15 19:58:43.154530: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-15 19:58:43.154543: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kuba): /proc/driver/nvidia/version does not exist
2022-01-15 19:58:43.154753: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
from utils import prepare_text
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig


VOCAB_SIZE = 1000
encoder = keras.layers.TextVectorization(
    standardize = prepare_text, 
    max_tokens=VOCAB_SIZE)
encoder.adapt(ds_train.map(lambda text, label: text))
encoder.get_vocabulary()


['',
 '[UNK]',
 'i',
 'to',
 'the',
 'a',
 'my',
 'and',
 'is',
 'you',
 'for',
 'in',
 'of',
 'it',
 'on',
 'have',
 'so',
 'but',
 'that',
 'me',
 'just',
 "i'm",
 'at',
 'be',
 'with',
 'not',
 'was',
 'this',
 'good',
 'get',
 '-',
 'up',
 'are',
 'out',
 'like',
 'go',
 'all',
 "it's",
 'day',
 'no',
 'your',
 'now',
 'going',
 'do',
 'got',
 'work',
 'love',
 'from',
 'am',
 'about',
 "don't",
 'will',
 'what',
 'we',
 'back',
 'some',
 'had',
 'can',
 'new',
 'one',
 'too',
 'really',
 'im',
 'still',
 'if',
 'time',
 "can't",
 'its',
 'u',
 'today',
 'how',
 'know',
 'see',
 'as',
 'want',
 'when',
 'more',
 'think',
 'last',
 'has',
 'an',
 '&amp;',
 'they',
 'hope',
 'need',
 'lol',
 'been',
 'off',
 '2',
 'morning',
 'then',
 'or',
 'miss',
 'thanks',
 'much',
 'there',
 'only',
 'wish',
 'oh',
 'very',
 'he',
 'home',
 'twitter',
 'by',
 'would',
 'well',
 'her',
 'feel',
 'here',
 'getting',
 'great',
 'why',
 'make',
 "i'll",
 'night',
 'happy',
 'could',
 'should',
 'did

In [6]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])


In [7]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])


In [8]:
history = model.fit(ds_train, epochs=10,
                    validation_data=ds_test,
                    validation_steps=30)


Epoch 1/10
1000/1000 [==============================] - 19s 15ms/step - loss: 0.6214 - accuracy: 0.6101 - val_loss: 0.5488 - val_accuracy: 0.7021
Epoch 2/10
1000/1000 [==============================] - 13s 13ms/step - loss: 0.5251 - accuracy: 0.7241 - val_loss: 0.5212 - val_accuracy: 0.7208
Epoch 3/10
1000/1000 [==============================] - 13s 13ms/step - loss: 0.5063 - accuracy: 0.7360 - val_loss: 0.5138 - val_accuracy: 0.7375
Epoch 4/10
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4991 - accuracy: 0.7405 - val_loss: 0.5107 - val_accuracy: 0.7365
Epoch 5/10
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4947 - accuracy: 0.7425 - val_loss: 0.5088 - val_accuracy: 0.7396
Epoch 6/10
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4912 - accuracy: 0.7458 - val_loss: 0.5070 - val_accuracy: 0.7458
Epoch 7/10
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4876 - accuracy: 0.7483 - val_loss: 0.5052 -

In [9]:
model.save("model1")

2022-01-15 20:02:14.952170: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: model1/assets


INFO:tensorflow:Assets written to: model1/assets
